# THIS DOES NOT WORK

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import libraries
from qick import *
from qick import asm_v2
from pickle_parser import *
import Pyro4
%pylab inline

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib

QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ


In [778]:
prog = QickProgram(soccfg) # Create program object

prog.declare_gen(ch=1, nqz=1)
prog.declare_gen(ch=1, nqz=1)
prog.set_pulse_registers(ch=1, gain=10000, freq=prog.freq2reg(100), phase=0, style="const", length=500)
prog.set_pulse_registers(ch=0, gain=10000, freq=prog.freq2reg(100), phase=0, style="const", length=500)
prog.declare_readout(ch=1,length=100,freq=100,gen_ch=1)
prog.pulse(ch=1, t=0)
prog.pulse(ch=0, t=0)
prog.trigger(adcs=[1], adc_trig_offset=0)
prog.waiti(0,1000)

prog.read(1,0,"lower",3)
prog.regwi(0,2,1)
prog.condj(0,2,'>',3,'after_reset')
prog.trigger(pins=[0], t=0, width=10)

prog.label('after_reset')
prog.wait_all()
prog.end()

prog.config_all(soc) # Load assembly to board
soc.tproc.start() # Run program
print(prog)


// Program

             regwi 1, $22, 43690667;            //freq = 43690667
             regwi 1, $23, 0;                   //phase = 0
             regwi 1, $25, 10000;               //gain = 10000
             regwi 1, $26, 590324;              //phrst| stdysel | mode | | outsel = 0b01001 | length = 500 
             regwi 0, $22, 43690667;            //freq = 43690667
             regwi 0, $23, 0;                   //phase = 0
             regwi 0, $25, 10000;               //gain = 10000
             regwi 0, $26, 590324;              //phrst| stdysel | mode | | outsel = 0b01001 | length = 500 
             regwi 1, $27, 0;                   //t = 0
             set 1, 1, $22, $23, $0, $25, $26, $27;//ch = 1, pulse @t = $27
             regwi 0, $27, 0;                   //t = 0
             set 0, 0, $22, $23, $0, $25, $26, $27;//ch = 0, pulse @t = $27
             regwi 0, $16, 32768;               //out = 0b1000000000000000
             seti 7, 0, $16, 0;                 //ch